In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from openai import OpenAI
from google import genai
from google.genai import types
import anthropic

# Specify the path to your .env file
dotenv_path = "/mnt/4d4f90e5-f220-481e-8701-f0a546491c35/arquivos/projetos/.env"
load_dotenv(dotenv_path=dotenv_path)

# Access and store the environment variable
xai_api_key = os.getenv("XAI_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
claude_api_key = os.getenv("ANTHROPIC_API_KEY")

# Config client
client_grok = OpenAI(api_key=xai_api_key, base_url="https://api.x.ai/v1")
client_gpt = OpenAI(api_key=openai_api_key)
client_gemini = genai.Client(api_key=google_api_key)
client_ds = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")
client_claude = anthropic.Anthropic(api_key=claude_api_key)

# Model
model_grok = 'grok-beta'
model_gpt = 'gpt-4.5-preview-2025-02-27'
model_gemini = 'gemini-2.0-flash-thinking-exp'
model_ds = 'deepseek-chat'
model_claude = 'claude-3-7-sonnet-20250219'


In [2]:
punctual_text = "刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在那江畔上，一江水冷月光满城的汪洋，我在时间的树下等了你很久，尘凡儿缠我谤我笑我白了头，你看那天边追逐落日的纸鸢，像一盏回首道别夤夜的风灯，我的心似流沙放逐在车辙旁，他日你若再返必颠沛在世上，若遇那秋夜雨倦鸟也淋淋，那却是花墙下弥留的枯黄，君住在钱塘东，妾在临安北，君去时褐衣红，小奴家腰上黄，寻差了罗盘经，错投在泉亭，奴辗转到杭城，君又生余杭。”"

In [3]:
def translate_and_retranslate(client, model, text):
    """
    Translates the given Chinese text to English and then back to Chinese using the specified client and model.

    Args:
        client: The client object (OpenAI, genai, or anthropic) to use for translation.
        model: The model to use for translation.
        text: The Chinese text to translate.

    Returns:
        A dictionary containing the model name, original text, English translation, and retranslated Chinese text.
    """

    # Step 1: Translate from ZH to EN
    prompt_zh_en = f"Translate the following Chinese text to English, providing only the translated text without any additional explanations or context: {text}"
    
    if isinstance(client, OpenAI):
        response_zh_en = client.chat.completions.create(model=model, messages=[{"role": "user", "content": prompt_zh_en}])
        generated_text_zh_en = response_zh_en.choices[0].message.content
    elif isinstance(client, genai.Client):
        response_zh_en = client.models.generate_content(model=model, contents=prompt_zh_en)
        generated_text_zh_en = response_zh_en.text
    elif isinstance(client, anthropic.Anthropic):
        response_zh_en = client.messages.create(model=model, max_tokens=1000, messages=[{"role": "user", "content": prompt_zh_en}])
        generated_text_zh_en = response_zh_en.content[0].text
    else:
        raise ValueError("Unsupported client type.")

    # Step 2: Translate back from EN to ZH
    prompt_en_zh = f"Translate the following English text to Chinese, providing only the translated text without any additional explanations or context: {generated_text_zh_en}"

    if isinstance(client, OpenAI):
        response_en_zh = client.chat.completions.create(model=model, messages=[{"role": "user", "content": prompt_en_zh}])
        generated_text_en_zh = response_en_zh.choices[0].message.content
    elif isinstance(client, genai.Client):
        response_en_zh = client.models.generate_content(model=model, contents=prompt_en_zh)
        generated_text_en_zh = response_en_zh.text
    elif isinstance(client, anthropic.Anthropic):
        response_en_zh = client.messages.create(model=model, max_tokens=1000, messages=[{"role": "user", "content": prompt_en_zh}])
        generated_text_en_zh = response_en_zh.content[0].text
    else:
        raise ValueError("Unsupported client type.")

    data = {
        'model': [model],
        'original_text': [text],
        'zh_en': [generated_text_zh_en],
        'en_zh': [generated_text_en_zh]
    }
    return data


In [4]:
models = {
    'gemini': {'model': model_gemini, 'client': client_gemini, 'data': None},
    'deepseek': {'model': model_ds, 'client': client_ds, 'data': None},
    'claude': {'model': model_claude, 'client': client_claude, 'data': None},
    'grok': {'model': model_grok, 'client': client_grok, 'data': None},
    'gpt': {'model': model_gpt, 'client': client_gpt, 'data': None}
}

for model_name, model_data in models.items():
    print(f"Translating and retranslating using model: {model_name}")
    model_data['data'] = translate_and_retranslate(model_data['client'], model_data['model'], punctual_text)
    print(f"Translation and retranslation complete for model: {model_name}")

Translating and retranslating using model: gemini
Translation and retranslation complete for model: gemini
Translating and retranslating using model: deepseek
Translation and retranslation complete for model: deepseek
Translating and retranslating using model: claude
Translation and retranslation complete for model: claude
Translating and retranslating using model: grok
Translation and retranslation complete for model: grok
Translating and retranslating using model: gpt
Translation and retranslation complete for model: gpt


In [21]:
df_list = []
for model_name, model_data in models.items():
    df = pd.DataFrame(model_data['data'])
    # Remove newline characters and excess spaces from all string columns
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.replace('\n', '', regex=False).str.replace(r'\s{2,}', ' ', regex=True).str.strip()
    df_list.append(df)

final_df = pd.concat(df_list, ignore_index=True)
print(final_df)


                           model  \
0  gemini-2.0-flash-thinking-exp   
1                  deepseek-chat   
2     claude-3-7-sonnet-20250219   
3                      grok-beta   
4     gpt-4.5-preview-2025-02-27   

                                       original_text  \
0  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
1  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
2  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
3  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
4  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   

                                               zh_en  \
0  I am the wandering tear on the rings of time,Y...   
1  Daolang, excerpt from "Flower Demon": "I am th...   
2  Dao Lang, partial lyrics of "Flower Demon":"I ...   
3  Here is the translation:Dao Lang, lyrics from ...   
4  Dao Lang, lyrics excerpt from "Hua Yao":"I am ...   

                                               en_zh  
0  我是时间轮环上漂泊的泪滴，你依然能在风中嗅到胭脂的气味。倘若我将誓言刻在那河岸上，一条冰冷的...  

In [22]:
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import jieba


In [30]:
# Tokenization function for Chinese texts using Jieba
def tokenize_chinese_jieba(text):
    # Use Jieba to cut the Chinese text into words
    return list(jieba.cut(text))

# Function to calculate BLEU (using unigram and bigram)
def calculate_bleu(candidate_tokens, reference_tokens, weights=(0.5, 0.5, 0.0, 0.0)):
    # We use weights (0.5, 0.5) for unigrams and bigrams
    try:
        return sentence_bleu([reference_tokens], candidate_tokens, weights=weights)
    except Exception as e:
        print(f"Error calculating BLEU: {e}")
        return 0

# Function to calculate CHRF (character n-gram F-score)
def calculate_chrf(candidate_tokens, reference_tokens):
    candidate_str = "".join(candidate_tokens)
    reference_str = "".join(reference_tokens)
    # Calculates the ratio between the intersection and the union of the characters of the reference
    return len(set(candidate_str) & set(reference_str)) / len(set(reference_str)) if len(set(reference_str)) > 0 else 0

# Function to calculate TER using TF-IDF vectorization and mean squared error
def calculate_ter(candidate_text, reference_text):
    vectorizer = TfidfVectorizer() #token_pattern=r"(?u)\b\w+\b" - Removed token pattern because it is not needed for chinese
    try:
        tfidf_matrix = vectorizer.fit_transform([candidate_text, reference_text])
        return mean_squared_error(tfidf_matrix[0].toarray(), tfidf_matrix[1].toarray())
    except Exception as e:
        print(f"Error calculating TER: {e}")
        return 0

# Function to calculate Semantic Similarity (TF-IDF + cosine)
def calculate_semantic_similarity(original, translated):
    vectorizer = TfidfVectorizer() #token_pattern=r"(?u)\b\w+\b" - Removed token pattern because it is not needed for chinese
    try:
        tfidf_matrix = vectorizer.fit_transform([original, translated])
        return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    except Exception as e:
        print(f"Error calculating Semantic Similarity: {e}")
        return 0

# Apply metrics to each row of the DataFrame
def calculate_metrics_for_row(row):
    original_text = row['original_text']
    en_zh = row['en_zh']

    # Tokenize the texts using Jieba
    original_tokens = tokenize_chinese_jieba(original_text)
    translated_tokens = tokenize_chinese_jieba(en_zh)

    # Calculate the metrics:
    bleu_value = calculate_bleu(translated_tokens, original_tokens)
    chrf_value = calculate_chrf(translated_tokens, original_tokens)
    ter_value = calculate_ter("".join(translated_tokens), "".join(original_tokens))
    semantic_similarity = calculate_semantic_similarity(original_text, en_zh)
    bleu_value_uniform = calculate_bleu(translated_tokens, original_tokens, weights=(0.25, 0.25, 0.25, 0.25))

    return pd.Series([bleu_value, bleu_value_uniform, chrf_value, ter_value, semantic_similarity])

def calculate_metrics_for_df(df):
    # tqdm.pandas(desc="Calculating metrics") # Removed progress bar
    return df.apply(calculate_metrics_for_row, axis=1)


In [32]:
# Assign the calculated metrics to new columns in the original DataFrame
metrics_df = calculate_metrics_for_df(final_df)
final_df[['BLEU', 'BLEU-Unif', 'CHRF', 'TER', 'Semantic Sim']] = metrics_df.reindex(final_df.index)
print(final_df)


                           model  \
0  gemini-2.0-flash-thinking-exp   
1                  deepseek-chat   
2     claude-3-7-sonnet-20250219   
3                      grok-beta   
4     gpt-4.5-preview-2025-02-27   

                                       original_text  \
0  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
1  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
2  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
3  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   
4  刀郎，《花妖》部分歌词：“我是那年轮上流浪的眼泪，你仍然能闻到风中的胭脂味，我若是将诺言刻在...   

                                               zh_en  \
0  I am the wandering tear on the rings of time,Y...   
1  Daolang, excerpt from "Flower Demon": "I am th...   
2  Dao Lang, partial lyrics of "Flower Demon":"I ...   
3  Here is the translation:Dao Lang, lyrics from ...   
4  Dao Lang, lyrics excerpt from "Hua Yao":"I am ...   

                                               en_zh      BLEU  BLEU-Unif  \
0  我是时间轮环上漂泊的泪滴，你依然能在风中嗅到胭脂的气味。倘

In [33]:
# Save the DataFrame to Excel with good formatting
final_df.to_excel('results_metrics/results_punctual.xlsx', index=False, engine='openpyxl')

# Apply formatting to the Excel file
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, PatternFill

# Load the workbook
wb = load_workbook('results_punctual.xlsx')
ws = wb.active

# Format headers
for cell in ws[1]:
    cell.font = Font(bold=True)
    cell.fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Auto-adjust column widths
for column in ws.columns:
    max_length = 0
    column_letter = column[0].column_letter
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column_letter].width = adjusted_width

# Save the formatted workbook
wb.save('results_punctual.xlsx')
print("Results saved to 'results_punctual.xlsx' with formatting")

FileNotFoundError: [Errno 2] No such file or directory: 'results_punctual.xlsx'